In [3]:
# For langchain
!pip install langchain-openai langchain-experimental -q

# For ArangoDB
!pip install python-arango -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 4.8 MB/s eta 0:00:00


In [32]:
!pip install matplotlib

     ---------------------------------------- 8.1/8.1 MB 17.2 MB/s eta 0:00:00
     ------------------------------------- 219.8/219.8 kB 13.1 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 23.3 MB/s eta 0:00:00
     ---------------------------------------- 72.0/72.0 kB ? eta 0:00:00
     ---------------------------------------- 107.7/107.7 kB ? eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os

os.environ["DATABASE_HOST"] = "https://b61c3b83bfe6.arangodb.cloud:8529"
os.environ["DATABASE_USERNAME"] = "root"
os.environ["DATABASE_PASSWORD"] = "RHr0KzkRUVlp61IisH8G"
os.environ["DATABASE_NAME"] = "DAC_devops_log"

In [2]:
from arango import ArangoClient
import networkx as nx
import nx_arangodb as nxadb

db = ArangoClient(hosts="https://b61c3b83bfe6.arangodb.cloud:8529") \
    .db(username="root",
        password="RHr0KzkRUVlp61IisH8G",
        name="DAC_devops_log",
        verify=True)

[16:12:31 -0500] [INFO]: NetworkX-cuGraph is unavailable: No module named 'cupy'.


In [10]:
task_dep_graph = nxadb.DiGraph(name="ds_tasks_dependence_graph")

[16:16:05 -0500] [INFO]: Graph 'ds_tasks_dependence_graph' exists.
[16:16:05 -0500] [INFO]: Default node type set to 'ds_tasks'


In [11]:
print(task_dep_graph.nodes)

['ds_tasks/TSK-1EA40351', 'ds_tasks/TSK-61053B2F', 'ds_tasks/TSK-DC598064', 'ds_tasks/TSK-F01DFD69', 'ds_tasks/TSK-A10D4E86', 'ds_tasks/TSK-3B211663', 'ds_tasks/TSK-9A5C2E8E', 'ds_tasks/TSK-B2BA63BA', 'ds_tasks/TSK-6D002AAE', 'ds_tasks/TSK-139E76E6', 'ds_tasks/TSK-1EC35081', 'ds_tasks/TSK-74DF4453', 'ds_tasks/TSK-4989D136', 'ds_tasks/TSK-D7B94EC9', 'ds_tasks/TSK-E23C633E', 'ds_tasks/TSK-855D95B6', 'ds_tasks/TSK-E5FE227A', 'ds_tasks/TSK-A9FC79CE', 'ds_tasks/TSK-A55F85D0', 'ds_tasks/TSK-1356205C', 'ds_tasks/TSK-B746A5B7', 'ds_tasks/TSK-92ACEA8C', 'ds_tasks/TSK-9073F198', 'ds_tasks/TSK-34501DAC', 'ds_tasks/TSK-5510D3A9', 'ds_tasks/TSK-F34D1882', 'ds_tasks/TSK-4EBD67FD', 'ds_tasks/TSK-0D0D0F73', 'ds_tasks/TSK-CE2CF100', 'ds_tasks/TSK-FD4B906A', 'ds_tasks/TSK-DACF3342', 'ds_tasks/TSK-CC092721', 'ds_tasks/TSK-A12A42C7', 'ds_tasks/TSK-8849435A', 'ds_tasks/TSK-5BC70421', 'ds_tasks/TSK-8B768931', 'ds_tasks/TSK-7DE2A58D', 'ds_tasks/TSK-7A51B62D', 'ds_tasks/TSK-97096B82', 'ds_tasks/TSK-FEDA370B',

In [8]:
task_col = db.collection("ds_tasks")

print(task_col.all())
for task in task_col.all():
    print(task)

<Cursor>
{'_key': 'TSK-48103DF8', '_id': 'ds_tasks/TSK-48103DF8', '_rev': '_jVHvXZ2---', 'TaskID': 'TSK-48103DF8', 'Description': 'Create a project plan for data migration', 'AssignedEmployees': '2,27,6', 'Advisors': '18', 'PrecedingTasks': '', 'StoryPoints': '2', 'StartTime': '2025-03-09', 'EstimatedFinishTime': '2025-03-10', 'Status': 'Planned', 'ActualFinishTime': ''}
{'_key': 'TSK-1EA40351', '_id': 'ds_tasks/TSK-1EA40351', '_rev': '_jVHvXZ2--A', 'TaskID': 'TSK-1EA40351', 'Description': 'Define data migration scope and objectives', 'AssignedEmployees': '16,34', 'Advisors': '7', 'PrecedingTasks': '', 'StoryPoints': '2', 'StartTime': '2025-03-08', 'EstimatedFinishTime': '2025-03-09', 'Status': 'Planned', 'ActualFinishTime': ''}
{'_key': 'TSK-97096B82', '_id': 'ds_tasks/TSK-97096B82', '_rev': '_jVHvXZ2--B', 'TaskID': 'TSK-97096B82', 'Description': 'Perform initial data analysis to understand current data architecture', 'AssignedEmployees': '31', 'Advisors': '17,24', 'PrecedingTasks': '

In [ ]:
import random


def assign_nodes(G):
    layers = nx.topological_generations(G.copy())

    assignment = {node: "Planned" for node in G.nodes}
    completed_probability = 1
    layers = [sorted(generation) for generation in layers]
    length = len(layers)
    for level, nodes in enumerate(layers):
        for node in nodes:
            bad = False
            for _, out_neighbor in G.out_edges(node):
                if assignment[out_neighbor] != "Completed":
                    bad = True
                    break
            if bad:
                continue
            if random.random() < completed_probability:
                if random.random() < 0.3:
                    assignment[node] = "Blocked"
                else:
                    assignment[node] = "In Progress"
            else:
                
                assignment[node] = "Completed"
        completed_probability -= 1 / (length)

    return assignment

from collections import Counter
assignment = assign_nodes(task_dep_graph)
print(assignment)
print(Counter(assignment.values()))


[18:50:24 -0500] [WARNING]: Note that copying a graph loses the connection to the database


{'ds_tasks/TSK-1EA40351': 'Completed', 'ds_tasks/TSK-61053B2F': 'Blocked', 'ds_tasks/TSK-DC598064': 'In Progress', 'ds_tasks/TSK-F01DFD69': 'Completed', 'ds_tasks/TSK-A10D4E86': 'Completed', 'ds_tasks/TSK-3B211663': 'Completed', 'ds_tasks/TSK-9A5C2E8E': 'In Progress', 'ds_tasks/TSK-B2BA63BA': 'Blocked', 'ds_tasks/TSK-6D002AAE': 'Blocked', 'ds_tasks/TSK-139E76E6': 'In Progress', 'ds_tasks/TSK-1EC35081': 'Blocked', 'ds_tasks/TSK-74DF4453': 'In Progress', 'ds_tasks/TSK-4989D136': 'Completed', 'ds_tasks/TSK-D7B94EC9': 'In Progress', 'ds_tasks/TSK-E23C633E': 'Blocked', 'ds_tasks/TSK-855D95B6': 'Blocked', 'ds_tasks/TSK-E5FE227A': 'Blocked', 'ds_tasks/TSK-A9FC79CE': 'Completed', 'ds_tasks/TSK-A55F85D0': 'Blocked', 'ds_tasks/TSK-1356205C': 'In Progress', 'ds_tasks/TSK-B746A5B7': 'In Progress', 'ds_tasks/TSK-92ACEA8C': 'Completed', 'ds_tasks/TSK-9073F198': 'Completed', 'ds_tasks/TSK-34501DAC': 'Completed', 'ds_tasks/TSK-5510D3A9': 'In Progress', 'ds_tasks/TSK-F34D1882': 'In Progress', 'ds_tasks

Counter({'In Progress': 22, 'Blocked': 16, 'Completed': 11})
